In [ ]:
# !pip install yfinance seaborn --upgrade
# !pip install --upgrade pandas
# !pip install jinja2
# !pip install plotly 

In [1]:
import streamlit as st
import yfinance as yf 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go 
from IPython.display import display 

c:\Users\Victor.moreira\AppData\Local\miniconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


**Overview DXY**

In [2]:
tickers_DXY = { 
    'DXY': 'DX-Y.NYB',
    'USDEUR': 'USDEUR=X',
    'USDGBP': 'USDGBP=X',
    'USDSEK': 'USDSEK=X',
    'USDJPY': 'USDJPY=X',
    'USDCHF': 'USDCHF=X',
    'USDCAD': 'USDCAD=X'
} 

weights_DXY = {
    'USDEUR': 57.6,
    'USDJPY': 13.6,
    'USDGBP': 11.9,
    'USDCAD': 9.1,
    'USDSEK': 4.2,
    'USDCHF': 3.6,
    'DXY': 100
}

windows = {
    'D1': 1,
    'Week': 5,
    '2 Weeks': 10,
    'Month': 21,
    '3 Months': 63,
    '6 Months': 126,
    '12 Months': 252
}

data_DXY = yf.download(list(tickers_DXY.values()),
                        period = '30y',
                        interval = '1d')['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  7 of 7 completed


In [3]:
df_DXY = pd.DataFrame()

for asset_DXY, ticker_DXY in tickers_DXY.items(): 
    df_DXY[asset_DXY] = data_DXY[ticker_DXY]

df_DXY1 = df_DXY[['DXY']].ffill()

df_DXY1['DXY_200d'] = df_DXY1['DXY'].rolling(window=200).mean()
df_DXY1['DXY_50d'] = df_DXY1['DXY'].rolling(window=50).mean()

df_plot_DXY = df_DXY1[['DXY','DXY_200d','DXY_50d']]


In [4]:
fig_dxy = go.Figure()

fig_dxy.add_trace(go.Scatter(
    x=df_plot_DXY.index,
    y=df_plot_DXY["DXY"],
    mode="lines",
    name="DXY",                      # Legenda na chart
    line=dict(color="black")          # Define a cor da linha
))

fig_dxy.add_trace(go.Scatter(
    x=df_plot_DXY.index,
    y=df_plot_DXY["DXY_200d"],
    mode="lines",
    name="200d Média",
    line=dict(color="red")           # Define a cor para a média móvel
))

fig_dxy.add_trace(go.Scatter(
    x=df_plot_DXY.index,
    y=df_plot_DXY["DXY_50d"],
    mode="lines",
    name="50d Média",
    line=dict(color="blue")           # Define a cor para a média móvel
))

fig_dxy.update_layout(
    title="Performance do DXY (10 anos)",
    yaxis=dict(range=[70, 130]),
    xaxis=dict(range=[
        df_plot_DXY.index[0], 
        df_plot_DXY.index[-1] + pd.Timedelta("40d")
    ])
)

In [5]:
assets_DXY = ['USDEUR', 'USDGBP', 'USDSEK', 'USDJPY', 'USDCHF', 'USDCAD', 'DXY']
heatmap_DXY_data = pd.DataFrame( index = assets_DXY, columns = windows.keys())

for asset_DXY in assets_DXY: 
    for label, w in windows.items():
        #Pega o valor mais recente e compara com w dias
        try: 
            recent_price = df_DXY[asset_DXY].iloc[-1]
            old_price = df_DXY[asset_DXY].iloc[-(1+w)]
            log_return = (np.log(recent_price) - np.log(old_price))*100
            heatmap_DXY_data.loc[asset_DXY,label] = log_return
        except:
            heatmap_DXY_data.loc[asset_DXY,label] = np.nan


# Adicionar pesos
heatmap_DXY_data['Weight (%)'] = heatmap_DXY_data.index.map(weights_DXY)
cols = ['Weight (%)'] + [c for c in heatmap_DXY_data.columns if c != 'Weight (%)']
heatmap_DXY_data = heatmap_DXY_data[cols]

# Formatação
cols_for_color = [c for c in heatmap_DXY_data.columns if c != 'Weight (%)']
for c in cols_for_color:
    heatmap_DXY_data[c] = pd.to_numeric(heatmap_DXY_data[c], errors='coerce')
abs_max = np.abs(heatmap_DXY_data[cols_for_color].values).max()

In [6]:

styled_dxy = (
            heatmap_DXY_data.style
            .set_properties(**{'text-align': 'center'})  # alinha o conteúdo das células
            .set_table_styles([
               {'selector': 'th', 'props': [('text-align','center')]},  # cabeçalhos
               {'selector': 'td', 'props': [('text-align','center')]}   # células
            ])
            .background_gradient(cmap='RdBu', subset=cols_for_color, vmin=-abs_max, vmax=abs_max)
            .format("{:+.2f}%", subset=cols_for_color)
            .format("{:.1f}%", subset=['Weight (%)'])
)

styled_dxy

,Weight (%),D1,Week,2 Weeks,Month,3 Months,6 Months,12 Months
USDEUR,57.6%,+0.53%,-3.31%,-4.20%,-3.62%,-9.16%,-4.10%,-5.31%
USDGBP,11.9%,-1.07%,-3.78%,-2.33%,-2.26%,-7.72%,-1.60%,-5.97%
USDSEK,4.2%,+1.17%,-1.87%,-1.59%,-2.42%,-12.02%,-6.48%,-9.62%
USDJPY,13.6%,-0.37%,-3.18%,-4.61%,-3.73%,-8.75%,-4.73%,-8.11%
USDCHF,3.6%,+0.52%,-4.45%,-7.21%,-7.24%,-10.37%,-5.17%,-10.62%
USDCAD,9.1%,+0.65%,-1.94%,-3.01%,-2.86%,-2.56%,+1.22%,+1.88%
DXY,100.0%,+0.57%,-2.71%,-3.96%,-3.11%,-8.37%,-3.22%,-5.24%


**Overview EME**

In [7]:
tickers_EME = {
    'USDCNY' : 'CNY=X',
    'USDMXN' : 'MXN=X',
    'USDKRW' : 'KRW=X',
    'USDINR' : 'INR=X',
    'USDBRL' : 'BRL=X',
    'USDTWD' : 'TWD=X',
    'USDSGD' : 'SGD=X',
    'USDHKD' : 'HKD=X',
    'USDVND' : 'VND=X',
    'USDMYR' : 'MYR=X',
    'USDTHB' : 'THB=X',
    'USDILS' : 'ILS=X',
    'USDIDR' : 'IDR=X',
    'USDPHP' : 'PHP=X',
    'USDCLP' : 'CLP=X',
    'USDCOP' : 'COP=X',
    'USDSAR' : 'SAR=X', 
    'USDARS' : 'ARS=X',
    'USDRUB' : 'RUB=X'
}
    
weights_EME = {
    'USDCNY' : 31.3,
    'USDMXN' : 25.7,
    'USDKRW' : 6.6,
    'USDINR' : 5.3,
    'USDBRL' : 3.9,
    'USDTWD' : 3.8,
    'USDSGD' : 3.1,
    'USDHKD' : 2.9,
    'USDVND' : 2.6,
    'USDMYR' : 2.4,
    'USDTHB' : 2.2,
    'USDILS' : 2.1,
    'USDIDR' : 1.3,
    'USDPHP' : 1.3,
    'USDCLP' : 1.2,
    'USDCOP' : 1.1,
    'USDSAR' : 1.1, 
    'USDARS' : 1.1,
    'USDRUB' : 1,
    'EME_Index' : 100
}

windows = {
        'D1': 1,
        'Week': 5,
        '2 Weeks': 10,
        'Month': 21,
        '3 Months': 63,
        '6 Months': 126,
        '12 Months': 252
}

data = yf.download(tickers=list(tickers_EME.values()),
                       period='19y',
                       interval='1d')['Close']

df_EME = pd.DataFrame()

for asset, ticker in tickers_EME.items(): 
    df_EME[asset] = data[ticker]

[*********************100%***********************]  19 of 19 completed


In [8]:
# Exemplo: df_EME possui várias colunas, cada uma representando uma moeda, ex: ['USDCNY','USDMXN','...']
# threshold = 0.3 (30%)
threshold = 0.3

# 1) Para cada coluna de df_EME, calculamos variação diária e detectamos outliers
for col in df_EME.columns:
    # Calcula variação percentual diária
    ret = df_EME[col].pct_change()
    
    # Cria uma máscara booleana para dias com variação acima de +30% ou abaixo de -30%
    outliers = ret.abs() > threshold
    
    # Substitui no df_EME o valor do dia 't' por NaN quando ret(t) é outlier
    # (df.loc[linhas, col] = np.nan)
    df_EME.loc[outliers, col] = np.nan

# 2) Fazer forward fill ou outra correção
# Substitui as ocorrências de NaN pelo valor anterior
df_EME = df_EME.ffill()

C:\Users\Victor.moreira\AppData\Local\Temp\ipykernel_18240\2265143809.py:8: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\Victor.moreira\AppData\Local\Temp\ipykernel_18240\2265143809.py:8: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\Victor.moreira\AppData\Local\Temp\ipykernel_18240\2265143809.py:8: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\Victor.moreira\AppData\Local

In [9]:
## Construindo o índice EME via lógica Logarítmica (média geométrica)
#1) Calculo log(prices) de cada par
#2) Calculo o log-return a cada hora: Ln(Pt) - Ln(Pt-1)
#3) Somo os log-returns com os devidos pesos normalizados
#4) Soma cumulativa para obter ln(índice)
#5) Exponenciar para ter o valor do índice 



#1)  pesos normalizados (para somarem 1.0)
total_weight = sum(weights_EME.values())
weights_norm = {k: v/total_weight for k, v in weights_EME.items()}

#2) log dos preços (df do mesmo shape de df_EME)
log_prices = np.log(df_EME)

#3) Log-returns horário (diferença horária)
log_returns = log_prices.diff()

#4) Soma ponderada dos log-returns a cada hora
# Criar uma Series "weighted_log_ret" que, para cada timestamp, some w_i*log_returns[i]
weighted_log_ret = pd.Series(0.0, index = log_returns.index)
for col in log_returns.columns:
    w = weights_norm[col]
    weighted_log_ret += w * log_returns[col]


#5) Índice do dólar vs. EME (cumulativo de log return, base 100)
#   EME_index(0) = 100 => EME_index(t) = 100*exp(soma cumulativa de log_return até t)
log_index = weighted_log_ret.cumsum()
eme_index = 100 * np.exp(log_index)

df_EME['EME_Index'] = eme_index
# Agora "eme_index" é uma Series com o valor do índice do dólar vs. essas moedas EME ao longo do tempo (horário)

df_EME1 = df_EME[['EME_Index']].ffill()

df_EME1['EME_Index_200d'] = df_EME1['EME_Index'].rolling(window=200).mean()
df_EME1['EME_Index_50d'] = df_EME1['EME_Index'].rolling(window=50).mean()

df_plot_EME = df_EME1[['EME_Index','EME_Index_200d','EME_Index_50d']]

In [10]:
fig_EME = go.Figure()

fig_EME.add_trace(go.Scatter(
    x=df_plot_EME.index,
    y=df_plot_EME["EME_Index"],
    mode="lines",
    name="EME",                      # Legenda na chart
    line=dict(color="black")          # Define a cor da linha
))

fig_EME.add_trace(go.Scatter(
    x=df_plot_EME.index,
    y=df_plot_EME["EME_Index_200d"],
    mode="lines",
    name="200d Média",
    line=dict(color="red")           # Define a cor para a média móvel
))

fig_EME.add_trace(go.Scatter(
    x=df_plot_EME.index,
    y=df_plot_EME["EME_Index_50d"],
    mode="lines",
    name="50d Média",
    line=dict(color="blue")           # Define a cor para a média móvel
))

fig_EME.update_layout(
    title="Performance do EME (10 anos)",
    yaxis=dict(range=[85, 145]),
    xaxis=dict(range=[
        df_plot_EME.index[0], 
        df_plot_EME.index[-1] + pd.Timedelta("40d")
    ])
)



In [11]:
# Montando heatmap
assets_EME = ['USDCNY','USDMXN','USDKRW','USDINR','USDBRL','USDTWD','USDSGD','USDHKD','USDVND','USDMYR','USDTHB','USDILS','USDIDR','USDPHP','USDCLP','USDCOP','USDSAR','USDARS','USDRUB','EME_Index']
heatmap_EME_data = pd.DataFrame(index=assets_EME, columns=windows.keys())

for asset_EME in assets_EME: 
    for label, w in windows.items():
        #Pega o valor mais recente e compara com w dias
        try: 
            recent_price = df_EME[asset_EME].iloc[-1]
            old_price = df_EME[asset_EME].iloc[-(1+w)]
            log_return = (np.log(recent_price) - np.log(old_price))*100
            # ret_simples = (np.exp(log_return)-1)*100
            # change_pct = ((recent_price / old_price) - 1)*100
            heatmap_EME_data.loc[asset_EME,label] = log_return
        except:
            heatmap_EME_data.loc[asset_EME,label] = np.nan



# Adicionar pesos
heatmap_EME_data['Weight (%)'] = heatmap_EME_data.index.map(weights_EME)
cols = ['Weight (%)'] + [c for c in heatmap_EME_data.columns if c != 'Weight (%)']
heatmap_EME_data = heatmap_EME_data[cols]

# Formatação
cols_for_color = [c for c in heatmap_EME_data.columns if c != 'Weight (%)']
for c in cols_for_color:
    heatmap_EME_data[c] = pd.to_numeric(heatmap_EME_data[c], errors='coerce')
abs_max = np.abs(heatmap_EME_data[cols_for_color].values).max()


In [12]:

styled_EME = (
            heatmap_EME_data.style
            .set_properties(**{'text-align': 'center'})  # alinha o conteúdo das células
            .set_table_styles([
               {'selector': 'th', 'props': [('text-align','center')]},  # cabeçalhos
               {'selector': 'td', 'props': [('text-align','center')]}   # células
            ])
            .background_gradient(cmap='RdBu', subset=cols_for_color, vmin=-abs_max, vmax=abs_max)
            .format("{:+.2f}%", subset=cols_for_color)
            .format("{:.1f}%", subset=['Weight (%)'])
)

styled_EME

,Weight (%),D1,Week,2 Weeks,Month,3 Months,6 Months,12 Months
USDCNY,31.3%,+0.31%,+0.09%,+0.79%,+1.07%,-0.21%,+2.65%,+0.97%
USDMXN,25.7%,-0.89%,-2.94%,-1.86%,+0.86%,-1.77%,+1.40%,+16.32%
USDKRW,6.6%,+0.69%,-2.84%,-2.97%,-1.41%,-0.59%,+4.20%,+3.70%
USDINR,5.3%,-0.62%,-0.42%,+0.23%,-1.48%,-0.89%,+1.90%,+2.75%
USDBRL,3.9%,+0.44%,-0.40%,+3.31%,+2.57%,-2.05%,+4.13%,+13.51%
USDTWD,3.8%,+0.41%,-1.48%,-2.26%,-1.32%,-0.92%,+1.32%,-0.41%
USDSGD,3.1%,-0.12%,-2.50%,-1.84%,-1.12%,-3.54%,+0.35%,-3.17%
USDHKD,2.9%,+0.03%,-0.14%,-0.28%,-0.20%,-0.35%,-0.21%,-0.94%
USDVND,2.6%,+0.29%,+0.04%,+0.88%,+1.13%,+1.62%,+2.39%,+1.50%
USDMYR,2.4%,-0.23%,-1.74%,-0.57%,-0.77%,-1.94%,+2.32%,-7.97%


***EME ex-China Index***

In [ ]:
    tickers_EME_ExC = {
    'USDMXN' : 'MXN=X',
    'USDKRW' : 'KRW=X',
    'USDINR' : 'INR=X',
    'USDBRL' : 'BRL=X',
    'USDTWD' : 'TWD=X',
    'USDSGD' : 'SGD=X',
    'USDHKD' : 'HKD=X',
    'USDVND' : 'VND=X',
    'USDMYR' : 'MYR=X',
    'USDTHB' : 'THB=X',
    'USDILS' : 'ILS=X',
    'USDIDR' : 'IDR=X',
    'USDPHP' : 'PHP=X',
    'USDCLP' : 'CLP=X',
    'USDCOP' : 'COP=X',
    'USDSAR' : 'SAR=X', 
    'USDARS' : 'ARS=X',
    'USDRUB' : 'RUB=X'
    }
    
    weights_EME_ExC = {
    'USDMXN' : 37.4,
    'USDKRW' : 9.6,
    'USDINR' : 7.7,
    'USDBRL' : 5.7,
    'USDTWD' : 5.5,
    'USDSGD' : 4.5,
    'USDHKD' : 4.2,
    'USDVND' : 3.8,
    'USDMYR' : 3.5,
    'USDTHB' : 3.2,
    'USDILS' : 3.1,
    'USDIDR' : 1.9,
    'USDPHP' : 1.9,
    'USDCLP' : 1.7,
    'USDCOP' : 1.6,
    'USDSAR' : 1.6, 
    'USDARS' : 1.6,
    'USDRUB' : 1.5,
    'EME_Index_ExC' : 100
    }

    windows = {
        'D1': 1,
        'Week': 5,
        '2 Weeks': 10,
        'Month': 21,
        '3 Months': 63,
        '6 Months': 126,
        '12 Months': 252
    }

    data_ExC = yf.download(tickers=list(tickers_EME_ExC.values()),
                       period='19y',
                       interval='1d')['Close']

    df_EME_ExC = pd.DataFrame()

    for asset, ticker in tickers_EME_ExC.items(): 
        df_EME_ExC[asset] = data_ExC[ticker]

In [ ]:
# Exemplo: df_EME_ExC possui várias colunas, cada uma representando uma moeda, ex: ['USDCNY','USDMXN','...']
# threshold = 0.3 (30%)
threshold = 0.2

# 1) Para cada coluna de df_EME_ExC, calculamos variação diária e detectamos outliers
for col in df_EME_ExC.columns:
    # Calcula variação percentual diária
    ret = df_EME_ExC[col].pct_change()
    
    # Cria uma máscara booleana para dias com variação acima de +30% ou abaixo de -30%
    outliers = ret.abs() > threshold
    
    # Substitui no df_EME_ExC o valor do dia 't' por NaN quando ret(t) é outlier
    # (df.loc[linhas, col] = np.nan)
    df_EME_ExC.loc[outliers, col] = np.nan

# 2) Fazer forward fill ou outra correção
# Substitui as ocorrências de NaN pelo valor anterior
df_EME_ExC = df_EME_ExC.ffill()

In [ ]:
## Construindo o índice EME via lógica Logarítmica (média geométrica)
#1) Calculo log(prices) de cada par
#2) Calculo o log-return a cada hora: Ln(Pt) - Ln(Pt-1)
#3) Somo os log-returns com os devidos pesos normalizados
#4) Soma cumulativa para obter ln(índice)
#5) Exponenciar para ter o valor do índice 



#1)  pesos normalizados (para somarem 1.0)
total_weight = sum(weights_EME_ExC.values())
weights_norm = {k: v/total_weight for k, v in weights_EME_ExC.items()}

#2) log dos preços (df do mesmo shape de df_EME)
log_prices = np.log(df_EME_ExC)

#3) Log-returns horário (diferença horária)
log_returns = log_prices.diff()

#4) Soma ponderada dos log-returns a cada hora
# Criar uma Series "weighted_log_ret" que, para cada timestamp, some w_i*log_returns[i]
weighted_log_ret = pd.Series(0.0, index = log_returns.index)
for col in log_returns.columns:
    w = weights_norm[col]
    weighted_log_ret += w * log_returns[col]


#5) Índice do dólar vs. EME (cumulativo de log return, base 100)
#   EME_index(0) = 100 => EME_index(t) = 100*exp(soma cumulativa de log_return até t)
log_index = weighted_log_ret.cumsum()
eme_ExC_index = 100 * np.exp(log_index)

df_EME_ExC['EME_Index_ExC'] = eme_ExC_index
# Agora "eme_index" é uma Series com o valor do índice do dólar vs. essas moedas EME ao longo do tempo (horário)

df_EME1_ExC = df_EME_ExC[['EME_Index_ExC']].ffill()

df_EME1_ExC['EME_Index_ExC_200d'] = df_EME1_ExC['EME_Index_ExC'].rolling(window=200).mean()
df_EME1_ExC['EME_Index_ExC_50d'] = df_EME1_ExC['EME_Index_ExC'].rolling(window=50).mean()

df_plot_EME_ExC = df_EME1_ExC[['EME_Index_ExC','EME_Index_ExC_200d','EME_Index_ExC_50d']]

In [ ]:
    fig_EME_ExC = go.Figure()

    fig_EME_ExC.add_trace(go.Scatter(
        x=df_plot_EME_ExC.index,
        y=df_plot_EME_ExC["EME_Index_ExC"],
        mode="lines",
        name="EME_ExC",                      # Legenda na chart
        line=dict(color="blue")          # Define a cor da linha
    ))

    fig_EME_ExC.add_trace(go.Scatter(
        x=df_plot_EME_ExC.index,
        y=df_plot_EME_ExC["EME_Index_ExC_200d"],
        mode="lines",
        name="200d Média",
        line=dict(color="red")           # Define a cor para a média móvel
    ))

    fig_EME_ExC.update_layout(
        title="Performance do EME_ExC (10 anos)",
        yaxis=dict(range=[85, 170]),
        xaxis=dict(range=[
                df_plot_EME_ExC.index[0], 
                df_plot_EME_ExC.index[-1] + pd.Timedelta("40d")
        ])
    )

***Dollar Broad***

In [ ]:
    tickers_BROAD = {
    'USDEUR' : 'EUR=X',
    'USDCNY' : 'CNY=X',
    'USDCAD' : 'CAD=X',
    'USDMXN' : 'MXN=X',
    'USDJPY' : 'JPY=X',
    'USDGBP' : 'GBP=X',
    'USDKRW' : 'KRW=X',
    'USDINR' : 'INR=X',
    'USDCHF' : 'CHF=X',
    'USDBRL' : 'BRL=X',
    'USDTWD' : 'TWD=X',
    'USDSGD' : 'SGD=X',
    'USDHKD' : 'HKD=X',
    'USDAUD' : 'AUD=X',
    'USDVND' : 'VND=X',
    'USDMYR' : 'MYR=X',
    'USDTHB' : 'THB=X',
    'USDILS' : 'ILS=X',
    'USDIDR' : 'IDR=X',
    'USDPHP' : 'PHP=X',
    'USDCLP' : 'CLP=X',
    'USDCOP' : 'COP=X',
    'USDSAR' : 'SAR=X', 
    'USDARS' : 'ARS=X',
    'USDRUB' : 'RUB=X',
    'USDSEK' : 'SEK=X',
    }
    
    weights_BROAD = {
    'USDEUR': 18.6,
    'USDCNY' : 16.2,
    'USDCAD': 13.6,
    'USDMXN' : 13.3,
    'USDJPY': 6.4,
    'USDGBP': 5.1,
    'USDKRW' : 3.4,
    'USDINR' : 2.7,
    'USDCHF': 2.7,
    'USDBRL' : 2,
    'USDTWD' : 2,
    'USDSGD' : 1.6,
    'USDHKD' : 1.5,
    'USDAUD' : 1.4,
    'USDVND' : 1.3,
    'USDMYR' : 1.3,
    'USDTHB' : 1.1,
    'USDILS' : 1.1,
    'USDIDR' : 0.7,
    'USDPHP' : 0.7,
    'USDCLP' : 0.6,
    'USDCOP' : 0.6,
    'USDSAR' : 0.6, 
    'USDARS' : 0.5,
    'USDRUB' : 0.5,
    'USDSEK': 0.5,
    'BROAD_Index' : 100
    }

    windows = {
        'D1': 1,
        'Week': 5,
        '2 Weeks': 10,
        'Month': 21,
        '3 Months': 63,
        '6 Months': 126,
        '12 Months': 252
    }

    data_BROAD = yf.download(tickers=list(tickers_BROAD.values()),
                       period='19y',
                       interval='1d')['Close']

    df_BROAD = pd.DataFrame()

    for asset_BROAD, ticker in tickers_BROAD.items(): 
        df_BROAD[asset_BROAD] = data_BROAD[ticker]

In [ ]:
# Exemplo: df_BROAD possui várias colunas, cada uma representando uma moeda, ex: ['USDCNY','USDMXN','...']
# threshold = 0.3 (30%)
threshold = 0.2

# 1) Para cada coluna de df_BROAD, calculamos variação diária e detectamos outliers
for col in df_BROAD.columns:
    # Calcula variação percentual diária
    ret = df_BROAD[col].pct_change()
    
    # Cria uma máscara booleana para dias com variação acima de +30% ou abaixo de -30%
    outliers = ret.abs() > threshold
    
    # Substitui no df_BROAD o valor do dia 't' por NaN quando ret(t) é outlier
    # (df.loc[linhas, col] = np.nan)
    df_BROAD.loc[outliers, col] = np.nan

# 2) Fazer forward fill ou outra correção
# Substitui as ocorrências de NaN pelo valor anterior
df_BROAD = df_BROAD.ffill()

In [ ]:
    ## Construindo o índice EME via lógica Logarítmica (média geométrica)
    #1) Calculo log(prices) de cada par
    #2) Calculo o log-return a cada hora: Ln(Pt) - Ln(Pt-1)
    #3) Somo os log-returns com os devidos pesos normalizados
    #4) Soma cumulativa para obter ln(índice)
    #5) Exponenciar para ter o valor do índice 



    #1)  pesos normalizados (para somarem 1.0)
    total_weight = sum(weights_BROAD.values())
    weights_norm = {k: v/total_weight for k, v in weights_BROAD.items()}

    #2) log dos preços (df do mesmo shape de df_EME)
    log_prices = np.log(df_BROAD)

    #3) Log-returns horário (diferença horária)
    log_returns = log_prices.diff()

    #4) Soma ponderada dos log-returns a cada hora
    # Criar uma Series "weighted_log_ret" que, para cada timestamp, some w_i*log_returns[i]
    weighted_log_ret = pd.Series(0.0, index = log_returns.index)
    for col in log_returns.columns:
        w = weights_norm[col]
        weighted_log_ret += w * log_returns[col]


    #5) Índice do dólar vs. EME (cumulativo de log return, base 100)
    #   EME_index(0) = 100 => EME_index(t) = 100*exp(soma cumulativa de log_return até t)
    log_index = weighted_log_ret.cumsum()
    BROAD_index = 100 * np.exp(log_index)

    df_BROAD['BROAD_Index'] = BROAD_index
    # Agora "eme_index" é uma Series com o valor do índice do dólar vs. essas moedas EME ao longo do tempo (horário)

    df_BROAD1 = df_BROAD[['BROAD_Index']].ffill()

    df_BROAD1['BROAD_Index_200d'] = df_BROAD1['BROAD_Index'].rolling(window=200).mean()
    df_BROAD1['BROAD_Index_50d'] = df_BROAD1['BROAD_Index'].rolling(window=50).mean()

    df_plot_BROAD = df_BROAD1[['BROAD_Index','BROAD_Index_200d','BROAD_Index_50d']]


In [ ]:
    fig_BROAD = go.Figure()

    fig_BROAD.add_trace(go.Scatter(
        x=df_plot_BROAD.index,
        y=df_plot_BROAD["BROAD_Index"],
        mode="lines",
        name="BROAD Dollar",                      # Legenda na chart
        line=dict(color="black")          # Define a cor da linha
    ))

    fig_BROAD.add_trace(go.Scatter(
        x=df_plot_BROAD.index,
        y=df_plot_BROAD["BROAD_Index_200d"],
        mode="lines",
        name="200d Média",
        line=dict(color="red")           # Define a cor para a média móvel
    ))

    fig_BROAD.add_trace(go.Scatter(
        x=df_plot_BROAD.index,
        y=df_plot_BROAD["BROAD_Index_50d"],
        mode="lines",
        name="50d Média",
        line=dict(color="blue")           # Define a cor para a média móvel
    ))

    fig_BROAD.update_layout(
        title="Performance do BROAD Dollar",
        yaxis=dict(range=[85, 120]),
        xaxis=dict(range=[
            df_plot_BROAD.index[0], 
            df_plot_BROAD.index[-1] + pd.Timedelta("40d")
        ])
    )

In [ ]:
    # Montando heatmap
    assets_BROAD = ['USDEUR','USDCNY','USDCAD','USDMXN','USDJPY','USDGBP','USDKRW','USDINR','USDCHF','USDBRL','USDTWD','USDSGD','USDHKD','USDAUD','USDVND','USDMYR','USDTHB','USDILS','USDIDR','USDPHP','USDCLP','USDCOP','USDSAR','USDARS','USDRUB','USDSEK','BROAD_Index']
    heatmap_BROAD_data = pd.DataFrame(index=assets_BROAD, columns=windows.keys())

    for asset_BROAD in assets_BROAD: 
        for label, w in windows.items():
            #Pega o valor mais recente e compara com w dias
            try: 
                recent_price = df_BROAD[asset_BROAD].iloc[-1]
                old_price = df_BROAD[asset_BROAD].iloc[-(1+w)]
                log_return = (np.log(recent_price) - np.log(old_price))*100
                # ret_simples = (np.exp(log_return)-1)*100
                # change_pct = ((recent_price / old_price) - 1)*100
                heatmap_BROAD_data.loc[asset_BROAD,label] = log_return
            except:
                heatmap_BROAD_data.loc[asset_BROAD,label] = np.nan

    # Adicionar pesos
    heatmap_BROAD_data['Weight (%)'] = heatmap_BROAD_data.index.map(weights_BROAD)
    cols = ['Weight (%)'] + [c for c in heatmap_BROAD_data.columns if c != 'Weight (%)']
    heatmap_BROAD_data = heatmap_BROAD_data[cols]

    # Formatação
    cols_for_color = [c for c in heatmap_BROAD_data.columns if c != 'Weight (%)']
    for c in cols_for_color:
        heatmap_BROAD_data[c] = pd.to_numeric(heatmap_BROAD_data[c], errors='coerce')
    abs_max = np.abs(heatmap_BROAD_data[cols_for_color].values).max()

In [ ]:
styled_BROAD = (
            heatmap_BROAD_data.style
            .set_properties(**{'text-align': 'center'})  # alinha o conteúdo das células
            .set_table_styles([
               {'selector': 'th', 'props': [('text-align','center')]},  # cabeçalhos
               {'selector': 'td', 'props': [('text-align','center')]}   # células
            ])
            .background_gradient(cmap='RdBu', subset=cols_for_color, vmin=-abs_max, vmax=abs_max)
            .format("{:+.2f}%", subset=cols_for_color)
            .format("{:.1f}%", subset=['Weight (%)'])
)

styled_BROAD